# Clip Networks

- Clip by boundary file
- Clip by geographic jurisdiction
- Clip by transit to roadway

In [15]:
from pathlib import Path
import geopandas as gpd
from network_wrangler import load_roadway_from_dir
from network_wrangler.roadway.clip import clip_roadway

- Example directory

In [ ]:
EXAMPLE_DIR = Path.cwd().parent / "examples"
STPAUL = EXAMPLE_DIR / "stpaul"
STPAUL.is_dir()

## Clip Roadway Network to Boundary

This section of the notebook recreates the logic in the script `/bin/clip_roadway.py`

1. Read in roadway network

In [ ]:
full_road_net = load_roadway_from_dir(STPAUL)

- Make sure the node loaded correctly

In [ ]:
mymap = full_road_net.nodes_df.sample(200).explore(
    color="grey", tiles="CartoDB positron", name="sampled original points"
)
mymap

2. Load a boundary

In [ ]:
BOUNDARY_DIR = Path.cwd().parent / "tests" / "data"
list(BOUNDARY_DIR.glob("*.geojson"))

In [ ]:
unionstation_gdf = gpd.read_file(BOUNDARY_DIR / "unionstation.geojson")
unionstation_gdf.explore(m=mymap, name="Union Station Boundary", color="red")

3. Clip network to boundary

In [ ]:
union_station_net = clip_roadway(full_road_net, boundary_gdf=unionstation_gdf)
union_station_net.nodes_df.explore(m=mymap, color="green")
union_station_net.links_df.explore(
    m=mymap, color="green", style_kwds={"opacity": 0.5, "weight": 20}
)

## Clip to a geocoded place name from Open Street Map

In [ ]:
summit_net = clip_roadway(full_road_net, boundary_geocode="Summit - University, St. Paul, MN")
summit_net.nodes_df.explore(m=mymap, color="blue")
summit_net.links_df.explore(
    m=mymap, color="blue", style_kwds={"opacity": 0.5, "weight": 20}
)

## Clip Transit Network to Roadway

`clip_transit` clips the transit to either a boundary or a roadway network.

The default is to only retain trips which have at least two stops within the boundary/network but
it is parameterized so that you can change it.

In [ ]:
from network_wrangler.transit.io import load_transit

stp_transit = load_transit(STPAUL)

from network_wrangler.transit.clip import clip_transit

union_station_transit = clip_transit(stp_transit, roadway_net=union_station_net)

In [ ]:
stp_transit.shape_links_gdf.explore(
    m=mymap, color="black", style_kwds={"opacity": 0.5, "weight": 10}
)
stp_transit.stops_gdf.explore(
    m=mymap, color="black", marker_kwds={"fill_opacity": 0.5, "line_opacity": 0.7}
)
union_station_transit.shape_links_gdf.explore(m=mymap, color="palegreen", style_kwds={"weight": 5})
union_station_transit.stops_gdf.explore(
    m=mymap, color="palegreen", marker_kwds={"fill_opacity": 0.5, "line_opacity": 0.7}
)

In [ ]:
union_station_transit.feed.shapes